<a href="https://colab.research.google.com/github/milver/Experiments/blob/main/PWS_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Wunderground PWS Stations test

### Dependencies

In [34]:
from bs4 import BeautifulSoup as bs
import requests
import math
import re

### Functions

In [35]:
# @title
def scrape(station, units):
  USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"
  LANGUAGE = "en-US,en;q=0.5"
  URL = "https://www.wunderground.com/dashboard/pws/"

  try:
      session = requests.Session()
      session.headers["User-Agent"] = USER_AGENT
      session.headers["Accept-Language"] = LANGUAGE
      session.headers["Content-Language"] = LANGUAGE
      html = session.get(URL + station["id"])

      soup = bs(html.text, "html.parser")
  except:
      print("error")
      return None
  data = {}


  data["LAST_UPDATED"] = soup.findAll(
      "span", attrs={"class": "ng-star-inserted"}, string=re.compile("updated")
  )[0].text
  print(soup.findAll(
      "span", attrs={"class": "ng-star-inserted"}, string=re.compile("updated")
  )[0].text)
  strings = data["LAST_UPDATED"].split()



  if (strings[0] == "(updated") and (strings[3] == "ago)"):
      value = int(strings[1])

      if (value >= 0) and (value <= 60):
          if strings[2][0:6] == "second":
              data["LAST_UPDATED"] = 1 / value

          elif strings[2][0:6] == "minute":
              data["LAST_UPDATED"] = 1 / (value * 60)

          elif strings[2][0:4] == "hour":
              if (value >= 0) and (value <= 24):
                  data["LAST_UPDATED"] = 1 / (value * 3600)

              else:
                  return None

          else:
              return None

      else:
          return None
  #print(strings)
  # Get Temperature
  if "temp" in station["parameters"]:
      # data["temp"] = soup.find("span", attrs={"class": "wu-value"})
      data["temp"] = soup.find(attrs={"class": "current-temp"})
      # print(data["temp"].text.replace('°', '').strip())

      data["temp"] = round(float(data["temp"].text.replace('°', '').strip()))

      if units["temp"] == "c":
          data["temp"] = round((data["temp"] - 32) * (5 / 9), 1)

  # Get Wind Speed
  if "wind_speed" in station["parameters"]:
      data["wind_speed"] = soup.findAll("span", attrs={"class": "wu-value"})
      #data["wind_speed"] = soup.findAll(attrs={"class": "weather__data weather__wind-gust"})
      print(data["wind_speed"])
      # the index is subject to change if they change the layout
      data["wind_speed"] = round(float(data["wind_speed"][8].text), 1)

      if units["speed"] == "kmph":
          data["wind_speed"] = round(data["wind_speed"] * 1.6, 1)

      elif units["speed"] == "mps":
          data["wind_speed"] = round(data["wind_speed"] * (4 / 9), 1)

  # Get Wind Gust
  if "wind_gust" in station["parameters"]:
      data["wind_gust"] = soup.findAll("span", attrs={"class": "wu-value"})
      data["wind_gust"] = round(float(data["wind_gust"][9].text), 1)

      if units["speed"] == "kmph":
          data["wind_gust"] = round(data["wind_gust"] * 1.6, 1)

      elif units["speed"] == "mps":
          data["wind_gust"] = round(data["wind_gust"] * (4 / 9), 1)

  # # Get Wind Direction
  # if "wind_direction" in station["parameters"]:
  #     data["wind_direction"] = soup.find(
  #         "span", attrs={"class": "text-bold"}
  #     ).text

  # Get Wind Bearing
  # if "wind_bearing" in station["parameters"]:
  #     data["wind_bearing"] = soup.find(
  #         "div", attrs={"class": "arrow-wrapper"}
  #     )

  #     string_full = ((data["wind_bearing"]["style"]).split())[1]
  #     string_start = string_full[0:7]
  #     string_end = string_full[-5:-1]

  #     if (string_start == "rotate(") and (string_end == "deg)"):
  #         data["wind_bearing"] = int(string_full[7:-5]) - 180
  #     else:
  #         data["wind_bearing"] = None

  # Get Precipitation Rate
  if "precip_rate" in station["parameters"]:
      data["precip_rate"] = soup.findAll("span", attrs={"class": "wu-value"})
      data["precip_rate"] = round(float(data["precip_rate"][11].text), 2)

      if units["precip"] == "mm":
          data["precip_rate"] = round(data["precip_rate"] * 25.4, 2)

  # Get Precipitation Total
  if "precip_total" in station["parameters"]:
      data["precip_total"] = soup.findAll("span", attrs={"class": "wu-value"})
      data["precip_total"] = round(float(data["precip_total"][14].text), 2)

      if units["precip"] == "mm":
          data["precip_total"] = round(data["precip_total"] * 25.4, 2)

  # Get Pressure
  if "pressure" in station["parameters"]:
      data["pressure"] = soup.findAll("span", attrs={"class": "wu-value"})
      data["pressure"] = round(float(data["pressure"][12].text), 2)

      if units["pressure"] == "hpa":
          data["pressure"] = round(data["pressure"] * 33.8637526, 2)

  # Get Humidity
  if "humidity" in station["parameters"]:
      data["humidity"] = soup.findAll("span", attrs={"class": "wu-value"})
      data["humidity"] = round(float(data["humidity"][13].text))

  # Get UV Index
  if "uv_index" in station["parameters"]:
      data["uv_index"] = soup.findAll("span", attrs={"class": "wu-value"})
      data["uv_index"] = round(float(data["uv_index"][15].text))

  # # Get Solar Radiation
  # if "radiation" in station["parameters"]:
  #     data["radiation"] = soup.findAll(
  #         "div", attrs={"class": "weather__text"}
  #     )
  #     strings = data["radiation"][-1].text.split()

  #     if strings[1][-8:-3] == "watts":
  #         data["radiation"] = round(float(strings[0]), 1)
  #     else:
  #         data["radiation"] = None

  station["data"] = data
  return station


In [36]:
# @title
def process(config):
    stations = []
    avg_values = {}
    for station in config["stations"]:
        stations.append(scrape(station, config["units"]))

    # Get average values, except wind bearing and wind direction
    for parameter in [
        "temp",
        "wind_speed",
        "wind_gust",
        "pressure",
        "humidity",
        "precip_rate",
        "precip_total",
        "uv_index",
        "radiation",
    ]:
        last_updated_total = 0
        for station in stations:
            if parameter in station["data"] and station["data"][parameter] != None:
                last_updated_total += station["data"]["LAST_UPDATED"]

        avg_values[parameter] = 0
        for station in stations:
            if parameter in station["data"] and station["data"][parameter] != None:
                last_updated_weight = (
                    station["data"]["LAST_UPDATED"] / last_updated_total
                )
                avg_values[parameter] += (
                    last_updated_weight * station["data"][parameter]
                )

        avg_values[parameter] = round(avg_values[parameter], 2)

    # Get average wind bearing
    # winds = []
    # wind_vectors = []
    # for station in stations:
    #     if (
    #         "wind_bearing" in station["data"]
    #         and station["data"]["wind_bearing"] != None
    #     ):
    #         winds.append(
    #             (station["data"]["wind_bearing"], station["data"]["wind_speed"])
    #         )

    # for i, wind in enumerate(winds):
    #     wind_vectors.append([None, None])
    #     wind_vectors[i][0] = math.sin(math.radians(wind[0])) * wind[1]
    #     wind_vectors[i][1] = math.cos(math.radians(wind[0])) * wind[1]

    # avg_x, avg_y = 0, 0
    # for wind_vector in wind_vectors:
    #     avg_x += wind_vector[0]
    #     avg_y += wind_vector[1]

    # avg_values["wind_bearing"] = round(math.degrees(math.atan2(avg_x, avg_y)))
    # avg_values["wind_speed"] = round(
    #     math.sqrt(avg_x ** 2 + avg_y ** 2) / len(wind_vectors), 2
    # )

    # if avg_values["wind_bearing"] < 0:
    #     avg_values["wind_bearing"] += 360

    return avg_values

### Config

In [37]:
# @title
if __name__ == "__main__":
    config = {
        "units": {
            "temp": "F",
            "pressure": "hpa",
            "speed": "mph",
            "precip": "in",
        },
        "stations": [
            {
                "id": "KVAASHBU142",
                "parameters": [
                    "temp",
                    "wind_speed",
                    "wind_gust",
                    "wind_bearing",
                    "pressure",
                    "humidity",
                    "precip_rate",
                    "precip_total",
                    "uv_index",
                    "radiation",
                ],
            },
            {
                "id": "KVABROAD56",
                "parameters": [
                    "temp",
                    "wind_speed",
                    "wind_gust",
                    "wind_bearing",
                    "pressure",
                    "humidity",
                    "precip_rate",
                    "precip_total",
                    "uv_index",
                    "radiation",
                ],
            },
            {
                "id": "KVAASHBU115",
                "parameters": [
                    "temp",
                    "wind_speed",
                    "wind_gust",
                    "wind_bearing",
                    "pressure",
                    "humidity",
                    "precip_rate",
                    "precip_total",
                ],
            },
        ],
    }

### Run

In [38]:
for k, v in process(config).items():
    print(f"{k}\t{v}")

(updated 3 minutes ago)
[<span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">64</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">70</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">78</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">61</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">72</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">65</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">70.2</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">70.6</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">0.0</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">0.0</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">63.0</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="col

In [39]:
print(config['stations'])

[{'id': 'KVAASHBU142', 'parameters': ['temp', 'wind_speed', 'wind_gust', 'wind_bearing', 'pressure', 'humidity', 'precip_rate', 'precip_total', 'uv_index', 'radiation'], 'data': {'LAST_UPDATED': 0.005555555555555556, 'temp': 70, 'wind_speed': 0.0, 'wind_gust': 0.0, 'precip_rate': 0.0, 'precip_total': 0.0, 'pressure': 1000.0, 'humidity': 78, 'uv_index': 2}}, {'id': 'KVABROAD56', 'parameters': ['temp', 'wind_speed', 'wind_gust', 'wind_bearing', 'pressure', 'humidity', 'precip_rate', 'precip_total', 'uv_index', 'radiation'], 'data': {'LAST_UPDATED': 0.08333333333333333, 'temp': 69, 'wind_speed': 0.0, 'wind_gust': 0.0, 'precip_rate': 0.0, 'precip_total': 0.0, 'pressure': 1011.17, 'humidity': 81, 'uv_index': 1}}, {'id': 'KVAASHBU115', 'parameters': ['temp', 'wind_speed', 'wind_gust', 'wind_bearing', 'pressure', 'humidity', 'precip_rate', 'precip_total'], 'data': {'LAST_UPDATED': 0.03125, 'temp': 70, 'wind_speed': 0.0, 'wind_gust': 0.0, 'precip_rate': 0.0, 'precip_total': 0.0, 'pressure': 10